In [1]:
import polars as pl
import altair as alt

In [2]:
import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [3]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1993
).filter(
    pl.col("komora_komplet").str.contains('Poslanecká') | pl.col('komora_komplet').str.contains('PČR, PS')
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).with_columns(
    pl.when(pl.col("soubor") == "1996ps_stenprot_006schuz_s006024.htm").then(pl.lit(2006)).otherwise(pl.col('obdobi')).alias('obdobi')
).with_columns(
    pl.col("text").str.split("(").list.slice(1).list.eval(
        pl.element().str.split(')').list.first()
    ).list.eval(
        pl.element().str.to_lowercase().str.strip_chars(), parallel=True
    ).alias("poznamky_text")
).with_columns(
    pl.col("mluvci").str.split("(").list.slice(1).list.eval(
        pl.element().str.split(')').list.first()
    ).list.eval(
        pl.element().str.to_lowercase().str.strip_chars(), parallel=True
    ).alias("poznamky_mluvci")
).drop_nulls(
    subset=['datum']
).collect()

In [4]:
df.sample(10)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Předseda PSP Lubomír Zaorálek""","""2002ps_stenprot_023schuz_s023394.htm""","""/sqw/detail.sqw?id=252""","""Zahajuji hlasování. Kdo je pro, stiskněte tlačítko a zvedněte ruku. Kdo je proti? V hlasování pořadové číslo 787 z přítomných 163 pro hlasovalo 141, p…",17,"""ps""",2002,23,394,2003-12-12,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""23. schůze""",true,true,true,"""252""",28,[],[]
"""Místopředseda PSP Pavel Tollner""","""1993ps_stenprot_017schuz_s017020.htm""","""/sqw/hlasy.sqw?G=7050""","""Děkuji. Ptám se sněmovny, zda si ještě někdo přeje vystoupit s interpelací nebo otázkou na členy vlády. Pokud tomu tak není, můžeme tento bod ukončit.…",17,"""ps""",1993,17,20,1994-03-24,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""17. schůze""",true,true,true,null,308,[],[]
"""Místopředseda PSP Karel Havlíček""","""2021ps_stenprot_035schuz_s035192.htm""","""/sqw/detail.sqw?id=6621""","""Děkuji a končím obecnou rozpravu. (Hlásí se poslanec Výborný.) Já myslím, že to stihneme. Končím obecnou rozpravu. Závěrečná slova? Nejsou. Zahajuji p…",6,"""ps""",2021,35,192,2022-09-27,"""PČR, PS 2021-...""","""35. schůze""",true,true,true,"""6621""",35,"[""hlásí se poslanec výborný."", ""neformální domlouvání.""]",[]
"""Místopředseda PSP Vojtěch Filip""","""2006ps_stenprot_011schuz_s011148.htm""","""/sqw/detail.sqw?id=303""","""Dobře. Jak jsem od začátku dnešního jednání shovívavost sama, tak předpokládám, že jde pouze o chybu v psaní. Pan předseda volební komise to odůvodní.""",7,"""ps""",2006,11,148,2007-02-07,"""Parlament České republiky, Poslanecká sněmovna 2006 – 2010""","""11. schůze""",true,true,true,"""303""",23,[],[]
"""Poslankyně Jitka Vojtilová""","""2002ps_stenprot_053schuz_s053361.htm""","""/sqw/detail.sqw?id=5284""","""L40.""",14,"""ps""",2002,53,361,2006-02-08,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""53. schůze""",true,true,true,"""5284""",0,[],[]
"""Místopředseda PSP Jan Bartošek""","""2013ps_stenprot_029schuz_s029072.htm""","""/sqw/detail.sqw?id=6178""","""Já vám děkuji. Další faktickou poznámku má nahlášenu pan poslanec Komárek. Máte slovo.""",12,"""ps""",2013,29,72,2015-06-17,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""29. schůze""",true,true,true,"""6178""",12,[],[]
"""Poslankyně Karla Maříková""","""2017ps_stenprot_087schuz_s087363.htm""","""/sqw/detail.sqw?id=6469""","""Děkuji za slovo, pane předsedající. Pane ministře, jestli mě můžete poslouchat... (Ministr Plaga hovoří s poslankyní Hyťhovou.) Pak můžete komunikovat…",3,"""ps""",2017,87,363,2021-03-25,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""87. schůze""",true,true,true,"""6469""",274,"[""ministr plaga hovoří s poslankyní hyťhovou."", ""se smíchem""]",[]
"""Místopředseda PSP Vojtěch Filip""","""2006ps_stenprot_032schuz_s032113.htm""","""/sqw/detail.sqw?id=303""","""To byl s přednostním právem pan místopředseda vlády Jiří Čunek. Nyní pan Cyril Svoboda. Já vás eviduji, kdo jste se přihlásili. Po Cyrilu Svobodovi řá…",4,"""ps""",2006,32,113,2008-06-06,"""Parlament České republiky, Poslanecká sněmovna 2006 – 2010""","""32. schůze""",true,true,true,"""303""",41,[],[]
"""Poslanec Jan Mládek""","""2002ps_stenprot_036schuz_s036041.htm""","""/sqw/detail.sqw?id=5269""","""Dámy a pánové, velmi se omlouvám, ale chtěl jsem hlasovat ano, ale nemám tam žádné hlasování. Zpochybňuji hlasování.""",4,"""ps""",2002,36,41,2004-10-13,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""36. schůze""",true,true,true,"""5269""",17,[],[]


In [5]:
df.group_by("pocet_slov").len().sort(by="pocet_slov")

pocet_slov,len
u32,u32
0,1353
1,7805
2,3233
3,4404
4,9252
5,7187
6,7199
7,7763
8,8473


In [6]:
df.select(pl.col("pocet_slov")).median()

pocet_slov
f64
45.0


In [7]:
df.select(pl.col("pocet_slov")).quantile(0.25)

pocet_slov
f64
18.0


In [8]:
slov_po_roce = df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('pocet_slov').sum())

In [9]:
len(df)

585336

In [10]:
df.columns

['mluvci',
 'soubor',
 'mluvci_id',
 'text',
 'poradi',
 'komora',
 'obdobi',
 'schuze',
 'cast',
 'datum',
 'komora_komplet',
 'schuze_komplet',
 'prepsano',
 'autorizovano',
 'mluvci_jisty',
 'id',
 'pocet_slov',
 'poznamky_text',
 'poznamky_mluvci']

In [11]:
def zavorka(retezec, doslova=True, frejm=df, epocha="datum", delka="1y"):

    frejm = frejm.with_columns(
        (pl.col('komora').cast(str) + pl.lit('-') + pl.col('obdobi').cast(str) + pl.lit('-') + pl.col('schuze').cast(str) + pl.lit('-') + pl.col('cast').cast(str) + pl.lit('-') + pl.col('poradi').cast(str)).alias('id')
    )

    prislusne_projevy = frejm.explode("poznamky_text").explode("poznamky_mluvci").filter(
            pl.col('poznamky_text').str.contains(retezec, literal=doslova) 
        |
            pl.col('poznamky_mluvci').str.contains(retezec, literal=doslova)
        ).unique(
        subset=['datum','cast','poradi']
        ).sort(by='datum')
    
    if epocha == "datum":
        
        projevu_celkem = frejm.sort(by='datum').group_by_dynamic(index_column='datum',every=delka).agg(pl.col("id").unique().len())
        
        prislusne_projevy = prislusne_projevy.group_by_dynamic(
            index_column='datum',every=delka
        ).agg(
            pl.col("id").unique().len()
        )

    elif epocha == "obdobi":

        projevu_celkem = frejm.group_by("obdobi").agg(pl.col("id").unique().len())
        prislusne_projevy = prislusne_projevy.group_by(
            "obdobi"
        ).agg(
            pl.col("id").unique().len()
        )

    return prislusne_projevy.join(
        projevu_celkem, how="right",on=epocha
    ).with_columns(
        (pl.col("id") / pl.col("id_right")).alias("podil")
    ).sort(by=epocha)

In [12]:
def zavorka(retezec, doslova=True, frejm=df, epocha="datum", delka="1y"):

    frejm = frejm.with_columns(
        (pl.col('komora').cast(str) + pl.lit('-') + pl.col('obdobi').cast(str) + pl.lit('-') + pl.col('schuze').cast(str) + pl.lit('-') + pl.col('cast').cast(str) + pl.lit('-') + pl.col('poradi').cast(str)).alias('id')
    )

    prislusne_projevy = frejm.explode("poznamky_text").explode("poznamky_mluvci").filter(
            pl.col('poznamky_text').str.contains(retezec, literal=doslova) 
        |
            pl.col('poznamky_mluvci').str.contains(retezec, literal=doslova)
        ).sort(by='datum')
    
    if epocha == "datum":
        
        projevu_celkem = frejm.sort(by='datum').group_by_dynamic(index_column='datum',every=delka).agg(pl.col("id").len())
        
        prislusne_projevy = prislusne_projevy.group_by_dynamic(
            index_column='datum',every=delka
        ).agg(
            pl.col("id").len()
        )

    elif epocha == "obdobi":

        projevu_celkem = frejm.group_by("obdobi").agg(pl.col("id").len())
        prislusne_projevy = prislusne_projevy.group_by(
            "obdobi"
        ).agg(
            pl.col("id").len()
        )

    return prislusne_projevy.join(
        projevu_celkem, how="right",on=epocha
    ).with_columns(
        (pl.col("id") / pl.col("id_right")).alias("podil")
    ).sort(by=epocha)

In [13]:
zavorka('smích',epocha='datum')

id,datum,id_right,podil
u32,date,u32,f64
40,1993-01-01,9366,0.004271
21,1994-01-01,7771,0.002702
40,1995-01-01,9237,0.00433
64,1996-01-01,9568,0.006689
58,1997-01-01,16632,0.003487
60,1998-01-01,11816,0.005078
95,1999-01-01,20885,0.004549
89,2000-01-01,24941,0.003568
90,2001-01-01,20971,0.004292


In [14]:
'; '.join(df.explode("poznamky_text").group_by(
    "poznamky_text"
).len(
).sort(
    by='len',
    descending=True
).head(500).filter(pl.col('poznamky_text').is_not_null()).select(pl.col('poznamky_text')).to_series().to_list())

'potlesk.; ?; ano.; souhlas.; nesouhlasné.; nesouhlas.; potlesk zprava.; hluk v sále.; ministr: nesouhlas.; slibuji.; nesrozumitelné; předsedající: čas.; smích v sále.; ministr: souhlas.; potlesk poslanců čssd.; potlesk poslanců ods.; potlesk zleva.; upozornění na čas; potlesk poslanců ksčm.; potlesk v části sálu.; souhlasné.; potlesk části poslanců.; předsedající upozorňuje na čas.; 14.40 hodin; 14.50 hodin; ne.; 15.20 hodin; 15.10 hodin; 15.00 hodin; 15.40 hodin; upozornění na čas.; 15.30 hodin; 15.50 hodin; velký hluk v sále.; ministr nedoporučuje.; v sále je velký hluk.; 16.10 hodin; smích.; potlesk z lavic ano.; 16.20 hodin; potlesk poslanců ano.; neutrální.; 17.10 hodin; 16.00 hodin; 17.20 hodin; pobavení v sále.; 16.40 hodin; 17.30 hodin; 16.30 hodin; 10.20 hodin; 10.10 hodin; 11.00 hodin; 17.50 hodin; 10.50 hodin; 16.50 hodin; 11.40 hodin; 11.10 hodin; 17.40 hodin; 14.30 hodin; 11.20 hodin; 11.30 hodin; 10.40 hodin; 10.30 hodin; jednání zahájeno v 9.00 hodin.; 11.50 hodin; 18.0

In [15]:
def graf_vyvoje(slovo, epocha="datum"):
    if epocha == "datum":
        osa_x = 'datum:T'
    elif epocha == "obdobi":
        osa_x = "obdobi:N"
    return alt.Chart(
        zavorka(slovo,epocha=epocha).to_pandas(),
        width = 800
    ).mark_bar(
    ).encode(
        alt.X(osa_x),
        alt.Y("podil:Q")
    )

In [16]:
graf_vyvoje("smích",epocha="datum")

alt.Chart(...)

In [17]:
graf_vyvoje("hluk",epocha="obdobi")

alt.Chart(...)

In [18]:
graf_vyvoje("bučení",epocha="obdobi")

alt.Chart(...)

In [19]:
graf_vyvoje("smích",epocha="obdobi")

alt.Chart(...)

In [20]:
graf_vyvoje("potlesk",epocha="obdobi")

alt.Chart(...)

In [21]:
alt.Chart(
    zavorka("smích").to_pandas(),
    width = 800
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

alt.Chart(...)

In [22]:
alt.Chart(
    zavorka("výkřiky").to_pandas(),
    width = 800
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

alt.Chart(...)

In [23]:
df.filter(pl.col("text").str.contains("[Bb]učení"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Ministr zdravotnictví ČR David Rath""","""2002ps_stenprot_055schuz_s055136.htm""","""http://wtd.vlada.cz/scripts/detail.php?id=9214""","""Děkuji. Je to myslím docela zase signifikantní a typické, když názor vyslovený těmi, kterých se to bytostně týká, tedy lékaři zaměstnanci a zdravotním…",15,"""ps""",2002,55,136,2006-04-21,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""55. schůze""",true,true,true,null,201,[],[]
"""Poslanec David Rath""","""2006ps_stenprot_027schuz_s027049.htm""","""/sqw/detail.sqw?id=5406""","""Dámy a pánové, pane předsedající, já bych prosil, abyste poučil o slušném chování poslance ODS, aby zde nebučeli. Nejsou, jak jsou zvyklí, v chlívě. Z…",3,"""ps""",2006,27,49,2008-01-30,"""Parlament České republiky, Poslanecká sněmovna 2006 – 2010""","""27. schůze""",true,true,true,"""5406""",426,[],[]
"""Místopředseda PSP Vojtěch Pikal""","""2017ps_stenprot_040schuz_s040102.htm""","""/sqw/detail.sqw?id=6531""","""Děkuji za slovo. Protože nemáme tu britskou tradici, tak tady neprobíhalo během předchozích projevů bučení a podobně, místo toho všeobecný nezájem pří…",7,"""ps""",2017,40,102,2020-01-23,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""40. schůze""",true,true,true,"""6531""",272,"[""potlesk poslanců pirátů.""]",[]
"""Předsedkyně PSP Markéta Pekarová Adamová""","""2021ps_stenprot_005schuz_s005019.htm""","""/sqw/detail.sqw?id=6147""","""Vzhledem k tomu, že program schůze nebyl schválen a nikdo další se s přednostním právem nehlásí, tak já tuto 5. schůzi Poslanecké sněmovny končím. (Po…",2,"""ps""",2021,5,19,2021-12-15,"""PČR, PS 2021-...""","""5. schůze""",true,true,true,"""6147""",77,"[""poslanci tlučou do lavic, ozývá se bučení a volání."", ""schůze skončila ve 12.14 hodin.""]",[]
"""Místopředseda PSP Jan Skopeček""","""2021ps_stenprot_009schuz_s009170.htm""","""/sqw/detail.sqw?id=6418""","""Děkuji za slovo. Vážené kolegyně, vážení kolegové, prostřednictvím paní předsedající bych chtěl reagovat na pana předsedu Okamuru. Jsem přesvědčen, že…",3,"""ps""",2021,9,170,2022-02-18,"""PČR, PS 2021-...""","""9. schůze""",true,true,true,"""6418""",119,"[""bučení v lavicích spd."", ""potlesk z řad koalice.""]",[]
"""Poslanec Jakub Michálek""","""2021ps_stenprot_055schuz_s055007.htm""","""/sqw/detail.sqw?id=6477""","""Děkuji za slovo. Vážená paní předsedkyně, vážené kolegyně, vážení kolegové, pojďme prosím nechat v rozpravě vystoupit i ministry, například tady pana …",9,"""ps""",2021,55,7,2023-02-28,"""PČR, PS 2021-...""","""55. schůze""",true,true,true,"""6477""",148,"[""nesouhlas a bučení poslanců ano.""]",[]


In [24]:
alt.Chart(
    zavorka("bučen").to_pandas(),
    width = 250
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

alt.Chart(...)

In [25]:
df.filter(pl.col("text").str.contains("(Buče",literal=True))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Místopředseda PSP Jan Skopeček""","""2021ps_stenprot_009schuz_s009170.htm""","""/sqw/detail.sqw?id=6418""","""Děkuji za slovo. Vážené kolegyně, vážení kolegové, prostřednictvím paní předsedající bych chtěl reagovat na pana předsedu Okamuru. Jsem přesvědčen, že…",3,"""ps""",2021,9,170,2022-02-18,"""PČR, PS 2021-...""","""9. schůze""",true,true,true,"""6418""",119,"[""bučení v lavicích spd."", ""potlesk z řad koalice.""]",[]


In [26]:
alt.Chart(
    zavorka("posmě[cš]",doslova=False).to_pandas(),
    width = 250
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

alt.Chart(...)

In [27]:
df.filter(
    pl.col("datum").dt.year().is_between(1940,1970)
).filter(
    pl.col("text").str.contains("(Smích",literal=True)
)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]


In [28]:
alt.Chart(
    pl.concat([zavorka("smích"), zavorka("potlesk"), zavorka("křik")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

alt.Chart(...)

In [29]:
alt.Chart(
    pl.concat([zavorka("potlesk zleva"), zavorka("potlesk zprava")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

alt.Chart(...)

In [30]:
def obrat(cislo):
    if cislo >= 1:
        return cislo
    if cislo < 1:
        return -(1/cislo)

In [31]:
obrat(0.1)

-10.0

In [32]:
pravolevo = zavorka(
    "(potlesk zleva|potlesk na levici|potlesk z levé|potlesk vlevo)",doslova=False,delka="3mo"
).with_columns(
    pl.col("podil").alias("zleva")
).select(
    pl.col(['datum','zleva'])
).join(
    zavorka(
        "(potlesk zprava|potlesk na pravici|potlesk z pravé|potlesk vpravo)", doslova=False,delka="3mo"
    ).with_columns(
        pl.col("podil").alias("zprava")
    ).select(
        pl.col(['datum','zprava'])
    ), on="datum", how="full"
).with_columns(
    (pl.col('zprava') / pl.col('zleva')).map_elements(obrat, return_dtype=pl.Float32).alias('pravo-levo')
).filter(
    pl.col('datum').dt.year() >= 1993
).with_columns(
    pl.when(
        pl.col("datum") >= pl.date(2021,10,1)
    ).then(
        pl.lit("současná Sněmovna")
    ).otherwise(
        pl.lit("předchozí")
    ).alias("ktera_snemovna")
)

pravolevo.drop_nulls().sort(by='pravo-levo')

datum,zleva,datum_right,zprava,pravo-levo,ktera_snemovna
date,f64,date,f64,f32,str
2012-10-01,0.014401,2012-10-01,0.003173,-4.538462,"""předchozí"""
2012-04-01,0.019123,2012-04-01,0.004218,-4.533333,"""předchozí"""
2024-07-01,0.005112,2024-07-01,0.00118,-4.333333,"""současná Sněmovna"""
2012-01-01,0.02066,2012-01-01,0.004934,-4.1875,"""předchozí"""
2010-07-01,0.013906,2010-07-01,0.003793,-3.666667,"""předchozí"""
2012-07-01,0.022715,2012-07-01,0.006294,-3.608696,"""předchozí"""
2024-04-01,0.006551,2024-04-01,0.002055,-3.1875,"""současná Sněmovna"""
2008-04-01,0.008605,2008-04-01,0.002746,-3.133333,"""předchozí"""
1998-04-01,0.000844,1998-04-01,0.000281,-3.0,"""předchozí"""


In [33]:
pravolevo.head(3)

datum,zleva,datum_right,zprava,pravo-levo,ktera_snemovna
date,f64,date,f64,f32,str
1993-01-01,null,1993-01-01,null,null,"""předchozí"""
1993-04-01,null,1993-04-01,null,null,"""předchozí"""
1993-07-01,null,1993-07-01,null,null,"""předchozí"""


In [34]:
pravolevo.tail(3)

datum,zleva,datum_right,zprava,pravo-levo,ktera_snemovna
date,f64,date,f64,f32,str
2025-01-01,0.003411,2025-01-01,0.001706,-2.0,"""současná Sněmovna"""
2025-04-01,0.002073,2025-04-01,0.002221,1.071429,"""současná Sněmovna"""
2025-07-01,0.005714,2025-07-01,null,null,"""současná Sněmovna"""


## Po 9 letech se víc tleská zleva!

In [184]:
graf_do_clanku = alt.Chart(
    pravolevo.to_pandas(),
    title=alt.Title(
        "Odkud zněl ve Sněmovně potlesk",
        subtitle=[
            'Hlasitější bývá opozice: zleva se obvykle tleská za vlády pravice, zprava',
            'za vlády levice. Rekordní výchylku, a to doprava, pozorujeme ve třetím',
            'čtvrtletí roku 2019 – čase masových demonstrací proti vládě Andreje Babiše.'
        ]
    ),
    width=330,
    height=330
).mark_bar(
    height=2.5
).encode(
    alt.Y('datum:T', axis=alt.Axis(title=None, tickCount=10)),
    alt.X('pravo-levo:Q',scale=alt.Scale(domain=[-45,45]), axis=alt.Axis(title="← častěji se tleská zleva ~ častěji se tleská zprava →", titleAlign='right', titleX=270, orient="top", labelExpr="replace(datum.label, '−', '') + '×'")),
    alt.Color("ktera_snemovna:N", legend=alt.Legend(
                  title=None,
                  direction="horizontal",
                  orient="top",
              ), scale=alt.Scale(range=['#6B8EAA','#D55F50']))
).configure_axisX(
    grid=True, domain=False
).configure_axisY(
    grid=True, domain=False
).configure_view(
    stroke='transparent'
).resolve_axis(
    x="independent",
    y="independent"
).resolve_scale(
    y="shared"
)

graf_do_clanku

alt.Chart(...)

In [186]:
me_to_neurazi(
    graf=graf_do_clanku, 
    soubor='odkud_potlesk', 
    kredity='vizualizace: iROZHLAS.cz | zdroj projevů: psp.cz', 
    slozka_na_serveru='grafy-stenoprotokoly'
)

<figure>
    <a href="https://data.irozhlas.cz/grafy-stenoprotokoly/odkud_potlesk.svg" target="_blank">
    <img src="https://data.irozhlas.cz/grafy-stenoprotokoly/odkud_potlesk.svg" width="100%" alt="Graf s titulkem „Odkud zněl ve Sněmovně potlesk“. Další texty by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [38]:
pravolevo_obdobi = zavorka(
    "(potlesk zleva|potlesk na levici|potlesk z levé|potlesk vlevo)", epocha="obdobi",doslova=False
).with_columns(
    pl.col("podil").alias("zleva")
).select(
    pl.col(['obdobi','zleva'])
).join(
    zavorka(
        "(potlesk zprava|potlesk na pravici|potlesk z pravé|potlesk vpravo)", epocha="obdobi", doslova=False
    ).with_columns(
        pl.col("podil").alias("zprava")
    ).select(
        pl.col(['obdobi','zprava'])
    ), on="obdobi", how="full"
).with_columns(
    (pl.col('zprava') / pl.col('zleva')).map_elements(obrat, return_dtype=pl.Float32).alias('pravo-levo')
)
pravolevo_obdobi

obdobi,zleva,obdobi_right,zprava,pravo-levo
i32,f64,i32,f64,f32
1993,0.0001,1993,0.000033,-3.0
1996,0.000877,1996,0.000552,-1.588235
1998,0.000954,1998,0.001165,1.220779
2002,0.000791,2002,0.003724,4.707692
2006,0.005242,2006,0.005102,-1.027304
2010,0.013336,2010,0.006377,-2.091216
2013,0.00421,2013,0.011711,2.78187
2017,0.001221,2017,0.007204,5.900901
2021,0.005561,2021,0.004237,-1.3125


In [39]:
alt.Chart(
    pravolevo_obdobi.to_pandas(),
    title=["„Potlesk zprava“ (> 0) a „potlesk zleva“ (< 0)","ve stenozáznamech PS PSP ČR"]
).mark_bar().encode(alt.Y('obdobi:N'),alt.X('pravo-levo:Q',axis=alt.Axis(title="<– víc se tleská zleva | víc se tleská zprava ->")))

alt.Chart(...)

## Počkat: kdo sedí vpravo, a kdo vlevo?

In [41]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Andrej Babiš'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

5

In [42]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Andrej Babiš'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

28

In [43]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Petr Fiala'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

8

In [44]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Petr Fiala'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

0

In [45]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Tomio Okamura'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

2

In [46]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Tomio Okamura'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

10

In [47]:
len(df.filter(
    (pl.col('mluvci').str.contains('Grebeníček'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

13

In [48]:
len(df.filter(
    (pl.col('mluvci').str.contains('Grebeníček'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

0

In [49]:
len(df.filter(
    (pl.col('mluvci').str.contains('Miroslav Sládek'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

0

In [50]:
len(df.filter(
    (pl.col('mluvci').str.contains('Miroslav Sládek'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

9

In [51]:
len(df.filter(
    (pl.col('mluvci').str.contains('Bohuslav Sobotka'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

47

In [52]:
len(df.filter(
    (pl.col('mluvci').str.contains('Bohuslav Sobotka'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

0

In [53]:
alt.Chart(
    pl.concat([zavorka("pobavení"), zavorka("veselost"), zavorka("veselí")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

alt.Chart(...)

In [54]:
alt.Chart(
    pl.concat([zavorka("\(Hluk"), zavorka("\(Ticho")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

alt.Chart(...)

In [55]:
alt.Chart(
    pl.concat([zavorka("(Smích", doslova=True), zavorka("(Veselí", doslova=True), zavorka("(Pobavení", doslova=True)]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

alt.Chart(...)

In [56]:
alt.Chart(
    df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len()).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('text:Q', axis=alt.Axis(title=None)),
)

alt.Chart(...)

In [57]:
df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len())

datum,text
date,u32
1993-01-01,9337
1994-01-01,7730
1995-01-01,9219
1996-01-01,9345
1997-01-01,16367
1998-01-01,11729
1999-01-01,20383
2000-01-01,23721
2001-01-01,20242


In [58]:
zebricek_potlesku = df.filter(
    pl.col("obdobi") == 2021
).explode(
    "poznamky_text"
).filter(
    pl.col("poznamky_text").str.contains("potlesk")
).group_by(
    "mluvci"
).len(
).sort(
    by="len",descending=True
)

zebricek_potlesku

mluvci,len
str,u32
"""Poslankyně Alena Schillerová""",594
"""Poslanec Andrej Babiš""",394
"""Poslanec Tomio Okamura""",200
"""Místopředseda PSP Karel Havlíček""",179
"""Poslanec Radek Vondráček""",147
"""Poslanec Patrik Nacher""",142
"""Poslanec Radim Fiala""",138
"""Poslankyně Berenika Peštová""",131
"""Poslanec Robert Králíček""",112


In [59]:
df.filter(
    pl.col("obdobi") == 2010
).explode(
    "poznamky_text"
).filter(
    pl.col("poznamky_text").str.contains("smích") 
    | 
    pl.col("poznamky_text").str.contains("pobavení") 
    | 
    pl.col("poznamky_text").str.contains("veselí") 
    | 
    (~pl.col("poznamky_text").str.contains("smíchem"))
).group_by(
    "mluvci"
).len(
).sort(
    by="len",descending=True
)

mluvci,len
str,u32
"""Místopředseda PSP Lubomír Zaorálek""",1868
"""Předsedkyně PSP Miroslava Němcová""",1844
"""Místopředsedkyně PSP Vlasta Parkanová""",531
"""Místopředsedkyně PSP Kateřina Klasnová""",476
"""Místopředseda PSP Jiří Oliva""",394
"""Poslanec Bohuslav Sobotka""",292
"""Ministr financí ČR Miroslav Kalousek""",251
"""Poslanec Jeroným Tejc""",220
"""Poslanec David Rath""",188


In [60]:
zebricek_smichu = df.filter(
    pl.col("obdobi") == 2021
).explode(
    "poznamky_text"
).filter(
    pl.col("poznamky_text").str.contains("smích") 
    | 
    pl.col("poznamky_text").str.contains("pobavení") 
    | 
    pl.col("poznamky_text").str.contains("veselí")
).group_by(
    "mluvci"
).len(
).sort(
    by="len",descending=True
)

zebricek_smichu

mluvci,len
str,u32
"""Poslanec Andrej Babiš""",322
"""Poslanec Tomio Okamura""",92
"""Místopředseda PSP Karel Havlíček""",72
"""Poslanec Patrik Nacher""",66
"""Místopředsedkyně PSP Věra Kovářová""",52
"""Místopředseda PSP Jan Skopeček""",47
"""Poslankyně Berenika Peštová""",45
"""Poslanec Radek Vondráček""",40
"""Poslanec Jakub Michálek""",38


In [61]:
zebricek_slov = df.filter(
    pl.col("obdobi") == 2021
).group_by(
    "mluvci"
).agg(
    pl.col("pocet_slov").sum()
)

zebricek_slov

mluvci,pocet_slov
str,u32
"""Poslanec Karel Havlíček""",9408
"""Poslanec Petr Gazdík""",522
"""Poslanec Ivan Jáč""",3598
"""Předsedající Radek Vondráček""",3591
"""Poslanec Josef Bernard""",23192
"""Poslankyně Jana Mračková Vildumetzová""",168798
"""Poslanec Bohuslav Svoboda""",8043
"""Poslanec Jan Kuchař""",20836
"""Ministr zemědělství ČR Zdeněk Nekula""",17326


In [62]:
zebricek_smichu.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).drop_nulls(
).sort(
    by="podil",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
1,"""Poslanec Josef Váňa""",122,0.008197
6,"""Prezident České republiky Miloš Zeman""",2533,0.002369
1,"""Senátor Ladislav Václavec""",532,0.00188
10,"""Poslanec Vít Vomáčka""",13931,0.000718
1,"""Poslanec Jan Farský""",1434,0.000697
1,"""Senátor Zdeněk Nytra""",1499,0.000667
1,"""Poslanec Zbyněk Stanjura""",1752,0.000571
15,"""Poslanec Jaroslav Foldyna""",28874,0.000519
4,"""Poslanec Bohuslav Svoboda""",8043,0.000497


In [63]:
df.filter(pl.col("obdobi") == 2021).filter(pl.col("mluvci").str.contains("Josef Bernard")).filter(pl.col("text").str.contains("směj"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]


In [64]:
zebricek_smichu.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).drop_nulls(
).filter(
    pl.col("pocet_slov") > 10000
).sort(
    by="podil",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
10,"""Poslanec Vít Vomáčka""",13931,0.000718
15,"""Poslanec Jaroslav Foldyna""",28874,0.000519
8,"""Poslanec Aleš Dufek""",22142,0.000361
12,"""Ministr spravedlnosti ČR Pavel Blažek""",36491,0.000329
322,"""Poslanec Andrej Babiš""",1103072,0.000292
24,"""Poslanec Richard Brabec""",90254,0.000266
6,"""Poslanec Jan Volný""",24851,0.000241
4,"""Poslanec Karel Sládeček""",16648,0.00024
14,"""Poslanec Milan Brázdil""",58974,0.000237


In [65]:
df.filter(pl.col("mluvci").str.contains("Ivan Bartoš")).select(pl.col("pocet_slov").sum())

pocet_slov
u32
352984


In [66]:
df.filter(pl.col("mluvci").str.contains("Ivan Bartoš")).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("směj"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,list[str]
"""Místopředseda vlády a ministr pro místní rozvoj ČR Ivan Bartoš""","""2021ps_stenprot_019schuz_s019175.htm""","""https://www.vlada.cz/cz/clenove-vlady/ivan-bartos-191704/""","""(Směje se.) Já jsem rád, že to takhle řešíme. Tak uděláme analýzu, které nejdůležitější legislativy kdy šly v předchozím volebním období poslaneckým n…",2,"""ps""",2021,19,175,2022-05-11,"""PČR, PS 2021-...""","""19. schůze""",true,true,true,null,91,"""směje se.""",[]


In [67]:
zebricek_smichu.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).filter(
    pl.col("len").is_null()
).sort(
    by="pocet_slov",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
null,"""Poslanec Vladimír Zlínský""",115356,null
null,"""Poslanec Igor Hendrych""",89885,null
null,"""Poslankyně Marie Pošarová""",71513,null
null,"""Poslankyně Klára Dostálová""",68324,null
null,"""Poslanec Radek Rozvoral""",64748,null
null,"""Poslankyně Ivana Mádlová""",57931,null
null,"""Poslankyně Jana Hanzlíková""",56081,null
null,"""Poslankyně Vladimíra Lesenská""",53453,null
null,"""Poslanec Josef Kott""",53022,null


In [68]:
df.filter(pl.col("obdobi") == 2021).filter(pl.col("mluvci").str.contains("Vladimír Zlínský")).filter(pl.col("text").str.contains("(?i)smích"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]


In [69]:
df.filter(pl.col("obdobi") == 2021).filter(pl.col("mluvci").str.contains("Vít Vomáčka")).filter(pl.col("text").str.contains("(?i)smích"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Poslanec Vít Vomáčka""","""2021ps_stenprot_009schuz_s009223.htm""","""/sqw/detail.sqw?id=6792""","""Musím zareagovat na pana kolegu. Nevím, jak dlouho jste starostoval, já starostuju jednatřicet let. Myslím si, že Kravaře, policie říká o tom, že Krav…",7,"""ps""",2021,9,223,2022-03-03,"""PČR, PS 2021-...""","""9. schůze""",true,true,true,"""6792""",64,"[""smích z lavic ano."", ""oživení a potlesk zleva.""]",[]
"""Poslanec Vít Vomáčka""","""2021ps_stenprot_079schuz_s079077.htm""","""/sqw/detail.sqw?id=6792""","""Vážení páni ministři, dámy a pánové, přiznám se, že nevystupuji, abych reagoval na svého ctěného pana kolegu, ale vystupuji kvůli tomu, že jsem tady n…",5,"""ps""",2021,79,77,2023-10-17,"""PČR, PS 2021-...""","""79. schůze""",true,true,true,"""6792""",120,"[""smích několika poslanců.""]",[]
"""Poslanec Vít Vomáčka""","""2021ps_stenprot_088schuz_s088077.htm""","""/sqw/detail.sqw?id=6792""","""Děkuji za slovo, vážený pane předsedající. Vážení páni ministři, dámy a pánové, já jsem se hlásil o slovo, když tady to začalo jiskřit mezi panem Kolo…",15,"""ps""",2021,88,77,2024-01-17,"""PČR, PS 2021-...""","""88. schůze""",true,true,true,"""6792""",137,"[""pobavení v sále."", ""smích v sále.""]",[]
"""Poslanec Vít Vomáčka""","""2021ps_stenprot_088schuz_s088232.htm""","""/sqw/detail.sqw?id=6792""","""Děkuji za slovo. Vážená paní předsedající, dámy a pánové, já jsem tady minule mluvil o tom, že tady vznikají určité dvojice, a přiznám se, že vždycky …",3,"""ps""",2021,88,232,2024-01-23,"""PČR, PS 2021-...""","""88. schůze""",true,true,true,"""6792""",136,"[""smích v levé části jednacího sálu."", ""smích v levé části jednacího sálu, malý potlesk v pravé části sálu."", ""pobavení v jednacím sále.""]",[]
"""Poslanec Vít Vomáčka""","""2021ps_stenprot_113schuz_s113082.htm""","""/sqw/detail.sqw?id=6792""","""Takže potřetí, paní Peštová, děkuju, že do této doby... Ale vidím, že se hlásíte, tak to asi začne střílet, ale já odstupuju, protože na vás nemám. Dě…",1,"""ps""",2021,113,82,2024-10-03,"""PČR, PS 2021-...""","""113. schůze""",true,true,false,"""6792""",31,"[""smích v levé části sálu.""]",[]


In [70]:
zebricek_potlesku.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).filter(
    pl.col("len").is_null()
).sort(
    by="pocet_slov",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
null,"""Poslanec Josef Kott""",53022,null
null,"""Ministr školství, mládeže a tělovýchovy ČR Mikuláš Bek""",49888,null
null,"""Ministr průmyslu a obchodu ČR Lukáš Vlček""",18076,null
null,"""Poslanec David Štolpa""",17001,null
null,"""Ministr pro místní rozvoj ČR Petr Kulhánek""",15043,null
null,"""Poslanec Matěj Ondřej Havel""",13488,null
null,"""Ministr školství, mládeže a tělovýchovy ČR Vladimír Balaš""",12668,null
null,"""Poslanec Petr Bendl""",12321,null
null,"""Poslanec Michael Rataj""",10725,null


In [71]:
zebricek_potlesku.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).drop_nulls(
).filter(
    pl.col("pocet_slov") > 31000
).sort(
    by="podil",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
112,"""Poslanec Robert Králíček""",79673,0.001406
38,"""Poslankyně Renata Oulehlová""",31971,0.001189
594,"""Poslankyně Alena Schillerová""",513521,0.001157
33,"""Poslanec Milan Feranec""",34981,0.000943
45,"""Poslanec Marek Benda""",50236,0.000896
29,"""Poslanec Tomáš Kohoutek""",34246,0.000847
23,"""Ministr spravedlnosti ČR Pavel Blažek""",36491,0.00063
41,"""Poslanec Kamal Farhan""",67206,0.00061
147,"""Poslanec Radek Vondráček""",245689,0.000598


In [72]:
df.filter(pl.col("mluvci").str.contains("Robert Králíček")).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("potlesk")).group_by("text").len().sort(by="len")

text,len
str,u32
"""Pokud už nebyla vláda schopna vetovat pakt jako celek, měla aspoň vyjednat výjimku z relokační povinnosti. Velkou příležitost k tomu měla, byla předse…",1
"""Já sem klidně budu chodit po každém vašem vystoupení, protože mě ty vaše ubohé útoky a lži už fakt jako vytáčí, tak vás klidně budu stínovat. (Potlesk…",1
"""Kapitola čtyři, mistr nesplněných slibů. Ministr vnitra Vít Rakušan při nástupu do funkce v roce 2021 prohlásil, že bezpečnostní sbory si zaslouží sta…",1
"""Já musím reagovat na pana Vomáčku, to jinak nejde, prostřednictvím (Poslanec Vomáčka z lavice: Prostřednictvím.) - já jsem ještě nedokončil větu, vy j…",1
"""No, tak to jsme si asi nerozuměli. Protože jestli se chceme zdržovat tím, že si tady budeme říkat, co funguje a co nefunguje, tak já tu mám z dnešního…",1
"""Děkuji, pane předsedající. Já bych krátce zareagoval na pana kolegu Lochmana. Mě vážně strašně mrzí, že jste tu nebyl v minulém volebním období, mě to…",1
"""Zpátky ale k samotné digitalizaci. Systém řízení byl pouze iluzí. Rozhodnutí nebyla zaznamenávána, dokumentace scházela a odpovědné osoby nebyly ident…",1
"""Vážený pane místopředsedo, vážená vládo, kolegyně kolegové, mně se strašně líbí, jak pan ministr Síkela přijde pozdě s něčím a ještě se tu pokouší zne…",1
"""Abych na něco nezapomněl... (Dívá se do podkladů.) Potom statistika. My nejsme schopni ověřit - a už na výboru pro veřejnou správu - nejsme schopni ov…",1


In [73]:
df.filter(pl.col("mluvci").str.contains("Josef Váňa"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Poslanec Josef Váňa""","""2021ps_stenprot_117schuz_s117088.htm""","""/sqw/detail.sqw?id=6946""","""Děkuji za slovo, paní předsedající. Dobrý den, kolegyně, kolegové, já bych se chtěl zeptat: ti, co navrhli tady tohle, kam jste zařadili koňaře? (Smíc…",6,"""ps""",2021,117,88,2024-11-08,"""PČR, PS 2021-...""","""117. schůze""",true,true,true,"""6946""",113,"[""smích v sále."", ""potlesk vlevo."", ""potlesk vlevo.""]",[]
"""Poslanec Josef Váňa""","""2021ps_stenprot_127schuz_s127021.htm""","""/sqw/detail.sqw?id=6946""","""Na sjetině mám pro, ale jsem proti, a zpochybňuji hlasování.""",4,"""ps""",2021,127,21,2025-01-21,"""PČR, PS 2021-...""","""127. schůze""",true,true,true,"""6946""",9,[],[]


In [74]:
df.filter(pl.col("mluvci").str.contains("Josef Váňa")).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("potlesk")).group_by("text").len().sort(by="len")

text,len
str,u32
"""Děkuji za slovo, paní předsedající. Dobrý den, kolegyně, kolegové, já bych se chtěl zeptat: ti, co navrhli tady tohle, kam jste zařadili koňaře? (Smíc…",2


In [75]:
df.filter(pl.col("mluvci").str.contains("Josef Váňa")).head(1).select(pl.col("soubor")).item()

'2021ps_stenprot_117schuz_s117088.htm'

In [76]:
df.filter(pl.col("mluvci").str.contains("Josef Váňa")).head(1).select(pl.col("text")).item()

'Děkuji za slovo, paní předsedající. Dobrý den, kolegyně, kolegové, já bych se chtěl zeptat: ti, co navrhli tady tohle, kam jste zařadili koňaře? (Smích v sále.) Prosím vás, tady tohle je povolání, které může dělat jenom blázen, to znamená soboty, neděle, dvanáct hodin denně. A řeknu vám hroznou skutečnost - dneska v šedesáti neznám jediného člověka, který by pracoval u koní, a nebyl invalidní důchodce nebo neměl zdravotní problémy. (Potlesk vlevo.) Moje paní má 57 let, a když jsem přišel domů a řekl jsem jí, že půjde za deset let do důchodu, tak mě málem vyhnala z baráku. Je to hrozné, ale prostě mně se ta vaše důchodová reforma vůbec nelíbí. Děkuji. (Potlesk vlevo.)'

In [77]:
df.filter(pl.col("mluvci").str.contains("Váňa"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Poslanec Miroslav Váňa""","""2002ps_stenprot_008schuz_s008195.htm""","""/sqw/detail.sqw?id=5292""","""Já zpochybňuji hlasování, jelikož na hlasovacím zařízení mám ""ano"", ale hlasoval jsem ""ne"".""",14,"""ps""",2002,8,195,2002-12-10,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""8. schůze""",true,true,true,"""5292""",12,[],[]
"""Poslanec Miroslav Váňa""","""2002ps_stenprot_008schuz_s008269.htm""","""/sqw/detail.sqw?id=5292""","""Vážený pane předsedo, vážená vládo, vážená Poslanecká sněmovno, výbor pro veřejnou správu, regionální rozvoj a životní prostředí na své 10. schůzi 3. …",3,"""ps""",2002,8,269,2002-12-17,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""8. schůze""",true,true,true,"""5292""",663,[],[]
"""Poslanec Miroslav Váňa""","""2002ps_stenprot_008schuz_s008270.htm""","""/sqw/detail.sqw?id=5292""","""(14.20 hodin) Druhý argument, který mě vede k tvrzení, které jsem zde učinil, je ta skutečnost, že doposud nejsou v dostatečné míře vytvořeny legisla…",1,"""ps""",2002,8,270,2002-12-17,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""8. schůze""",true,true,false,"""5292""",480,"[""14.20 hodin""]",[]
"""Poslanec Miroslav Váňa""","""2002ps_stenprot_008schuz_s008273.htm""","""/sqw/detail.sqw?id=5292""","""Vážený pane předsedo, vážená vládo, vážená Poslanecká sněmovno, procedura hlasování bude následující. V rámci podrobné rozpravy zazněly pozměňovací ná…",7,"""ps""",2002,8,273,2002-12-17,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""8. schůze""",true,true,true,"""5292""",61,[],[]
"""Poslanec Miroslav Váňa""","""2002ps_stenprot_016schuz_s016250.htm""","""/sqw/detail.sqw?id=5292""","""Vážené kolegyně a kolegové, jsem nucen zpochybnit své hlasování. Hlasoval jsem ano, ale na sestavě mám křížek.""",21,"""ps""",2002,16,250,2003-05-22,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""16. schůze""",true,true,true,"""5292""",16,[],[]
"""Poslanec Miroslav Váňa""","""2002ps_stenprot_018schuz_s018220.htm""","""/sqw/detail.sqw?id=401""","""Vážená paní místopředsedkyně, vážená vládo, vážená Poslanecká sněmovno. Výbor pro veřejnou správu, regionální rozvoj a životní prostředí na své 18. sc…",6,"""ps""",2002,18,220,2003-07-02,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""18. schůze""",true,true,true,"""401""",240,[],[]
"""Poslanec Miroslav Váňa""","""2002ps_stenprot_018schuz_s018221.htm""","""/sqw/detail.sqw?id=401""","""(16.40 hodin) Dovolte mi, dámy a pánové, abych z tohoto místa ocenil tu skutečnost, že ke konci roku 2002 došlo v plné míře k přesunu finančních pros…",1,"""ps""",2002,18,221,2003-07-02,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""18. schůze""",true,true,false,"""401""",546,"[""16.40 hodin"", ""?""]",[]
"""Poslanec Miroslav Váňa""","""2002ps_stenprot_018schuz_s018222.htm""","""/sqw/detail.sqw?id=5292""","""Mé závěrečné slovo je velmi stručné, jelikož principiálně se shoduji s panem ministrem. Musím konstatovat, že doprovodný návrh usnesení paní kolegyně …",9,"""ps""",2002,18,222,2003-07-02,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""18. schůze""",true,true,true,"""5292""",36,[],[]
"""Poslanec Miroslav Váňa""","""2002ps_stenprot_018schuz_s018223.htm""","""/sqw/detail.sqw?id=5292""","""(17.00 hodin)""",1,"""ps""",2002,18,223,2003-07-02,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""18. schůze""",true,true,false,"""5292""",1,"[""17.00 hodin""]",[]


In [78]:
df.filter(pl.col("mluvci").str.contains("Vladimír Zlínský")).filter(pl.col("text").str.contains("(?i)směj")).select(pl.col("text")).head(1).item()

'Vážený pane předsedající, děkuji za slovo. Vážené kolegyně a kolegové, vážený pane ministře a hlavně vážená paní ministryně, děkuji vám, že jste tady a vyslechnete si můj projev. Já se přiznám, že budu zkracovat. Já jsem zachytil signál, taky jsem zachytil signál od kolegů z ANO, hlavně od některých, nebudu je jmenovat, už se smějí, kteří s hrůzou v očích se na mě dívali, jak to dlouho bude trvat, ten projev můj, a co tady budu říkat. Tak já vám řeknu, o co všechno přijdete. Co jsem měl připraveno. Měl jsem připravený úžasný projev o energetice, kde jsem chtěl mluvit o fúzní energetice, o fúzních reaktorech, tak bohužel se to nedozvíte. Nebudete o tom nic vědět. Jenom tady řeknu, že pro fúzní energetiku bude třeba lithium, Takže to máme v Krušných horách. To je celkem dobrá zpráva. Pak jsem tady chtěl hovořit o úžasném fotosyntetickém řetězci, jak ten fotosyntetický řetězec je fantasticky odladěný a jak umožňuje zachytávat světlo ze Slunce s fúzní reakcí na Slunci. Tak to se taky nedo

In [79]:
df.filter(pl.col("obdobi") == 2021).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("(?i)bučen")).group_by("mluvci").len()

mluvci,len
str,u32
"""Poslanec Jakub Michálek""",1
"""Místopředseda PSP Jan Skopeček""",1
"""Předsedkyně PSP Markéta Pekarová Adamová""",1


In [80]:
df.filter(pl.col("obdobi") == 2021).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("(?i)kři[kč]")).group_by("mluvci").len().sort(by='len',descending=True)

mluvci,len
str,u32
"""Poslankyně Alena Schillerová""",17
"""Místopředseda PSP Jan Bartošek""",14
"""Předsedkyně PSP Markéta Pekarová Adamová""",9
"""Místopředsedkyně PSP Olga Richterová""",9
"""Poslanec Tomio Okamura""",8
"""Poslanec Jakub Michálek""",7
"""Poslanec Andrej Babiš""",6
"""Místopředseda PSP Karel Havlíček""",5
"""Místopředseda PSP Jan Skopeček""",5
